# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import numpy as np
from matplotlib import pyplot as plt

In [0]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Convolution2D, Flatten, Activation, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.utils import np_utils

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_examples = 50000

In [0]:
X1_train = []
X1_test = []
X2_train = []
X2_test = []
Y1_train = []
Y1_test = []
Y2_train = []
Y2_test = []

for ix in range(n_examples):
    if y_train[ix] < 5:
        # put data in set 1
        X1_train.append(x_train[ix]/255.0)
        Y1_train.append(y_train[ix])
    else:
        # put data in set 2
        X2_train.append(x_train[ix]/255.0)
        Y2_train.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        X1_test.append(x_test[ix]/255.0)
        Y1_test.append(y_test[ix])
    else:
        # put data in set 2
        X2_test.append(x_test[ix]/255.0)
        Y2_test.append(y_test[ix])

In [0]:
X1_train = np.asarray(X1_train).reshape((-1, 32, 32, 3))
X1_test = np.asarray(X1_test).reshape((-1, 32, 32, 3))
X2_train = np.asarray(X2_train).reshape((-1, 32, 32, 3))
X2_test = np.asarray(X2_test).reshape((-1, 32, 32, 3))

Y1_train = np_utils.to_categorical(np.asarray(Y1_train), num_classes=5)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), num_classes=5)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), num_classes=10)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), num_classes=10)

In [34]:
print (X1_train.shape, X1_test.shape)
print (Y1_train.shape, Y1_test.shape)

(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 5) (5000, 5)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras.constraints import maxnorm

In [38]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(16, (3, 3), activation='relu',  kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
dropout_9 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 16)        4624      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               1606144   
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)             

In [0]:
import time, datetime

In [40]:
start = datetime.datetime.now()
hist1 = model.fit(x1_train, y1_train, nb_epoch=10, shuffle=True, batch_size=100, validation_data=(x1_val, y1_val), verbose=2)

time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for first training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 57s - loss: 1.1673 - acc: 0.5153 - val_loss: 0.9500 - val_acc: 0.6296
Epoch 2/10
 - 56s - loss: 0.9071 - acc: 0.6338 - val_loss: 0.8625 - val_acc: 0.6608
Epoch 3/10
 - 57s - loss: 0.8227 - acc: 0.6770 - val_loss: 0.7971 - val_acc: 0.6872
Epoch 4/10
 - 56s - loss: 0.7528 - acc: 0.7040 - val_loss: 0.7430 - val_acc: 0.7152
Epoch 5/10
 - 57s - loss: 0.6760 - acc: 0.7395 - val_loss: 0.7012 - val_acc: 0.7310
Epoch 6/10
 - 57s - loss: 0.6168 - acc: 0.7642 - val_loss: 0.6845 - val_acc: 0.7440
Epoch 7/10
 - 56s - loss: 0.5574 - acc: 0.7853 - val_loss: 0.6691 - val_acc: 0.7514
Epoch 8/10
 - 56s - loss: 0.5090 - acc: 0.8082 - val_loss: 0.6838 - val_acc: 0.7496
Epoch 9/10
 - 56s - loss: 0.4607 - acc: 0.8235 - val_loss: 0.7218 - val_acc: 0.7412
Epoch 10/10
 - 56s - loss: 0.4208 - acc: 0.8451 - val_loss: 0.6766 - val_acc: 0.7594


 -------------------- 

Time taken for first training:  0:09:24.140712

 -------------------- 




### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
for l in model.layers[:6]:
    l.trainable = False   

In [43]:
trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
dropout_9 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 16)        4624      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               1606144   
_________________________________________________________________
dense_16 (Dense)             (None, 128)             

In [45]:
start = datetime.datetime.now()
hist2 = trans_model.fit(x2_train, y2_train, nb_epoch=10, shuffle=True, batch_size=100, validation_data=(x2_val, y2_val), verbose=2)
time_taken = datetime.datetime.now() - start
print ('\n'*2, '-'*20, '\n')
print ('Time taken for final training: ', time_taken)
print ('\n', '-'*20, '\n'*2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 17s - loss: 0.9483 - acc: 0.6578 - val_loss: 0.7893 - val_acc: 0.7032
Epoch 2/10
 - 17s - loss: 0.7667 - acc: 0.7081 - val_loss: 0.7620 - val_acc: 0.7090
Epoch 3/10
 - 17s - loss: 0.7366 - acc: 0.7173 - val_loss: 0.7482 - val_acc: 0.7164
Epoch 4/10
 - 17s - loss: 0.7178 - acc: 0.7253 - val_loss: 0.7310 - val_acc: 0.7226
Epoch 5/10
 - 18s - loss: 0.7008 - acc: 0.7313 - val_loss: 0.7378 - val_acc: 0.7196
Epoch 6/10
 - 18s - loss: 0.6864 - acc: 0.7379 - val_loss: 0.7312 - val_acc: 0.7238
Epoch 7/10
 - 17s - loss: 0.6696 - acc: 0.7437 - val_loss: 0.7153 - val_acc: 0.7272
Epoch 8/10
 - 17s - loss: 0.6554 - acc: 0.7501 - val_loss: 0.7062 - val_acc: 0.7290
Epoch 9/10
 - 17s - loss: 0.6357 - acc: 0.7566 - val_loss: 0.7113 - val_acc: 0.7282
Epoch 10/10
 - 17s - loss: 0.6241 - acc: 0.7613 - val_loss: 0.7001 - val_acc: 0.7320


 -------------------- 

Time taken for final training:  0:02:54.128089

 -------------------- 




### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
# The two models were trained on a CPU for just 10 epochs each.

# The first model trained for classes 0..4, achieved a validation accuracy of 75% after 10 epochs. However, the training time was 9 minutes 24 seconds.

# The second model trained for classes 5..9 used transfer learning to expedite the training process. It achieved an accuracy of 73.2% after 10 epochs. 
# More importantly the training time was down to just less than 3 minnutes.

# Therefore, transfer learning significantly brought down the time for processing of data. But the accuracy needs to be further tried out.

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [48]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [51]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [52]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [53]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
analyze = CountVectorizer().build_analyzer()
text = "A WONDERFUL test phrase!"
analyze(text)

['wonderful', 'test', 'phrase']

In [58]:
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35482)

In [59]:
count_vect.vocabulary_.get(u'algorithm')

4683

In [0]:
 from sklearn.feature_extraction.text import TfidfTransformer


In [62]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape




(2257, 35482)

In [63]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35482)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [65]:
clf = LogisticRegression().fit(X_train_tfidf, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [69]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
  print (doc, twenty_train.target_names[category])

God is love soc.religion.christian
OpenGL on the GPU is fast comp.graphics


In [0]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', LogisticRegression())])

In [80]:
_ = text_clf.fit(twenty_train.data, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [81]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories,shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)   

0.8868175765645806

In [82]:
from sklearn import metrics
print (metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.96      0.75      0.85       319
         comp.graphics       0.83      0.97      0.89       389
               sci.med       0.93      0.86      0.89       396
soc.religion.christian       0.87      0.94      0.90       398

              accuracy                           0.89      1502
             macro avg       0.90      0.88      0.88      1502
          weighted avg       0.89      0.89      0.89      1502

